In [3]:
# risk_factor_analysis.py
# Comprehensive analysis of suicide risk factors and demographic patterns

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

def load_main_data():
    """Load the main dataset"""
    try:
        df = pd.read_csv("../../Final/final_clean_dataset.csv")
        print(f"✅ Main data loaded: {len(df)} rows")
        return df
    except Exception as e:
        print(f"❌ Error loading main data: {e}")
        return None

def load_wdi_data():
    """Load and process WDI CSV data"""
    try:
        # Load WDI data
        wdi_df = pd.read_csv("../../Dataset/WDICSV-rev.csv", encoding='utf-8')
        print(f"✅ WDI data loaded: {len(wdi_df)} indicators")
        
        # Transform from wide to long format
        id_vars = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code']
        wdi_long = wdi_df.melt(id_vars=id_vars, 
                              var_name='Year', 
                              value_name='Value')
        
        # Convert Year to numeric and drop missing values
        wdi_long['Year'] = pd.to_numeric(wdi_long['Year'], errors='coerce')
        wdi_long = wdi_long.dropna(subset=['Year', 'Value'])
        
        # Pivot to get indicators as columns
        wdi_pivot = wdi_long.pivot_table(
            index=['Country Name', 'Year'],
            columns='Indicator Name',
            values='Value'
        ).reset_index()
        
        print(f"📊 WDI data processed: {wdi_pivot.shape[1]} indicators")
        return wdi_pivot
        
    except Exception as e:
        print(f"❌ Error loading WDI data: {e}")
        return None

def merge_datasets(main_df, wdi_df):
    """Merge main dataset with WDI indicators"""
    if wdi_df is None:
        return main_df
    
    # Merge on Country Name and Year
    merged_df = pd.merge(
        main_df,
        wdi_df,
        on=['Country Name', 'Year'],
        how='left',
        suffixes=('', '_wdi')
    )
    
    print(f"🔗 Datasets merged: {merged_df.shape}")
    return merged_df

def create_risk_factor_heatmap(df):
    """Create correlation heatmap of risk factors with suicide rates"""
    
    # Select potential risk factor columns
    risk_columns = [
        'HDI', 'GDP_per_capita', 'log_GDP_per_capita',
        'Suicide_rate', 'HDI_growth', 'Suicide_change'
    ]
    
    # Add WDI indicators if available
    wdi_indicators = [
        'Population ages 15-64 (% of total population)',
        'Unemployment, total (% of total labor force)',
        'Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)',
        'Gini index',
        'Life expectancy at birth, total (years)',
        'School enrollment, secondary (% gross)',
        'Urban population (% of total population)'
    ]
    
    available_columns = [col for col in risk_columns if col in df.columns]
    
    for indicator in wdi_indicators:
        if indicator in df.columns and df[indicator].notna().sum() > 50:
            available_columns.append(indicator)
    
    # Calculate correlations
    corr_data = df[available_columns].corr()
    suicide_correlations = corr_data['Suicide_rate'].drop('Suicide_rate').sort_values(key=abs, ascending=False)
    
    # Create heatmap
    fig = px.imshow(
        corr_data,
        text_auto=True,
        aspect="auto",
        color_continuous_scale="RdBu_r",
        title="Correlation Heatmap: Suicide Rate vs Risk Factors",
        width=1000,
        height=800
    )
    
    # Add annotations for top correlations
    top_correlations = suicide_correlations.head(10)
    
    return fig, top_correlations

def create_demographic_risk_profiles(df):
    """Create visualizations for demographic risk patterns"""
    
    # Create development-risk matrix
    df['Development_Risk_Category'] = pd.cut(
        df['HDI'],
        bins=[0, 0.55, 0.70, 0.85, 1.0],
        labels=['Low HDI', 'Medium HDI', 'High HDI', 'Very High HDI']
    )
    
    df['Risk_Level'] = pd.cut(
        df['Suicide_rate'],
        bins=[0, 5, 10, 15, df['Suicide_rate'].max()],
        labels=['Low Risk', 'Medium Risk', 'High Risk', 'Very High Risk']
    )
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Development vs Suicide Risk Matrix',
            'Suicide Rate Distribution by Development Level',
            'Risk Profile by Continent',
            'Top Risk Factors by Development Category'
        ],
        specs=[[{"type": "scatter"}, {"type": "box"}],
               [{"type": "bar"}, {"type": "bar"}]]
    )
    
    # 1. Development-Risk Matrix
    matrix_data = df.groupby(['Development_Risk_Category', 'Risk_Level']).size().unstack(fill_value=0)
    
    for risk_level in matrix_data.columns:
        fig.add_trace(
            go.Bar(
                x=matrix_data.index,
                y=matrix_data[risk_level],
                name=risk_level,
                text=matrix_data[risk_level],
                textposition='auto',
                hovertemplate='<b>%{x}</b><br>%{fullData.name}: %{y} countries<extra></extra>'
            ),
            row=1, col=1
        )
    
    fig.update_xaxes(title_text="Development Level", row=1, col=1)
    fig.update_yaxes(title_text="Number of Countries", row=1, col=1)
    
    # 2. Suicide Rate Distribution by Development Level
    for dev_level in df['Development_Risk_Category'].dropna().unique():
        dev_data = df[df['Development_Risk_Category'] == dev_level]['Suicide_rate'].dropna()
        fig.add_trace(
            go.Box(
                y=dev_data,
                name=dev_level,
                boxpoints='outliers',
                marker=dict(size=4),
                line=dict(width=2),
                showlegend=False,
                hovertemplate='<b>%{x}</b><br>Suicide Rate: %{y:.2f}<extra></extra>'
            ),
            row=1, col=2
        )
    
    fig.update_xaxes(title_text="Development Level", row=1, col=2)
    fig.update_yaxes(title_text="Suicide Rate", row=1, col=2)
    
    # 3. Risk Profile by Continent
    continent_risk = df.groupby(['continent', 'Risk_Level']).size().unstack(fill_value=0)
    continent_risk_pct = continent_risk.div(continent_risk.sum(axis=1), axis=0) * 100
    
    for risk_level in continent_risk_pct.columns:
        fig.add_trace(
            go.Bar(
                x=continent_risk_pct.index,
                y=continent_risk_pct[risk_level],
                name=risk_level,
                text=continent_risk_pct[risk_level].round(1),
                textposition='auto',
                showlegend=False,
                hovertemplate='<b>%{x}</b><br>%{fullData.name}: %{y:.1f}%<extra></extra>'
            ),
            row=2, col=1
        )
    
    fig.update_xaxes(title_text="Continent", row=2, col=1)
    fig.update_yaxes(title_text="Percentage (%)", row=2, col=1)
    
    # 4. Key Risk Indicators by Development Category
    risk_indicators = ['GDP_per_capita', 'HDI_growth']
    if 'Unemployment, total (% of total labor force)' in df.columns:
        risk_indicators.append('Unemployment, total (% of total labor force)')
    if 'Gini index' in df.columns:
        risk_indicators.append('Gini index')
    
    for indicator in risk_indicators:
        if indicator in df.columns:
            indicator_data = df.groupby('Development_Risk_Category')[indicator].mean().dropna()
            fig.add_trace(
                go.Bar(
                    x=indicator_data.index,
                    y=indicator_data.values,
                    name=indicator,
                    text=indicator_data.round(2),
                    textposition='auto',
                    showlegend=True,
                    hovertemplate='<b>%{x}</b><br>%{fullData.name}: %{y:.2f}<extra></extra>'
                ),
                row=2, col=2
            )
    
    fig.update_xaxes(title_text="Development Level", row=2, col=2)
    fig.update_yaxes(title_text="Indicator Value", row=2, col=2)
    
    fig.update_layout(
        height=1000,
        width=1200,
        title_text="Demographic Risk Profiles and Development Patterns",
        showlegend=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    return fig

def create_economic_social_analysis(df):
    """Analyze economic and social determinants of suicide risk"""
    
    # Select key economic and social indicators
    economic_indicators = [
        'GDP_per_capita', 'HDI', 'Suicide_rate'
    ]
    
    social_indicators = []
    if 'Unemployment, total (% of total labor force)' in df.columns:
        social_indicators.append('Unemployment, total (% of total labor force)')
    if 'Gini index' in df.columns:
        social_indicators.append('Gini index')
    if 'Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)' in df.columns:
        social_indicators.append('Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)')
    if 'Life expectancy at birth, total (years)' in df.columns:
        social_indicators.append('Life expectancy at birth, total (years)')
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Economic Development vs Suicide Rate',
            'Unemployment vs Suicide Rate',
            'Income Inequality vs Suicide Rate',
            'Poverty vs Suicide Rate'
        ],
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # 1. Economic Development vs Suicide Rate
    valid_data = df.dropna(subset=['GDP_per_capita', 'Suicide_rate'])
    fig.add_trace(
        go.Scatter(
            x=valid_data['GDP_per_capita'],
            y=valid_data['Suicide_rate'],
            mode='markers',
            marker=dict(
                size=8,
                color=valid_data['HDI'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="HDI")
            ),
            text=valid_data['Country Name'],
            hovertemplate='<b>%{text}</b><br>GDP: $%{x:,.0f}<br>Suicide: %{y:.2f}<br>HDI: %{marker.color:.3f}<extra></extra>',
            showlegend=False
        ),
        row=1, col=1
    )
    fig.update_xaxes(title_text="GDP per Capita (USD)", row=1, col=1, type="log")
    fig.update_yaxes(title_text="Suicide Rate", row=1, col=1)
    
    # 2. Unemployment vs Suicide Rate
    if 'Unemployment, total (% of total labor force)' in df.columns:
        valid_data = df.dropna(subset=['Unemployment, total (% of total labor force)', 'Suicide_rate'])
        fig.add_trace(
            go.Scatter(
                x=valid_data['Unemployment, total (% of total labor force)'],
                y=valid_data['Suicide_rate'],
                mode='markers',
                marker=dict(
                    size=8,
                    color=valid_data['HDI'],
                    colorscale='Viridis',
                    showscale=False
                ),
                text=valid_data['Country Name'],
                hovertemplate='<b>%{text}</b><br>Unemployment: %{x:.1f}%<br>Suicide: %{y:.2f}<extra></extra>',
                showlegend=False
            ),
            row=1, col=2
        )
        fig.update_xaxes(title_text="Unemployment Rate (%)", row=1, col=2)
        fig.update_yaxes(title_text="Suicide Rate", row=1, col=2)
    
    # 3. Income Inequality vs Suicide Rate
    if 'Gini index' in df.columns:
        valid_data = df.dropna(subset=['Gini index', 'Suicide_rate'])
        fig.add_trace(
            go.Scatter(
                x=valid_data['Gini index'],
                y=valid_data['Suicide_rate'],
                mode='markers',
                marker=dict(
                    size=8,
                    color=valid_data['HDI'],
                    colorscale='Viridis',
                    showscale=False
                ),
                text=valid_data['Country Name'],
                hovertemplate='<b>%{text}</b><br>Gini: %{x:.1f}<br>Suicide: %{y:.2f}<extra></extra>',
                showlegend=False
            ),
            row=2, col=1
        )
        fig.update_xaxes(title_text="Gini Index (Inequality)", row=2, col=1)
        fig.update_yaxes(title_text="Suicide Rate", row=2, col=1)
    
    # 4. Poverty vs Suicide Rate
    if 'Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)' in df.columns:
        valid_data = df.dropna(subset=['Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)', 'Suicide_rate'])
        fig.add_trace(
            go.Scatter(
                x=valid_data['Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)'],
                y=valid_data['Suicide_rate'],
                mode='markers',
                marker=dict(
                    size=8,
                    color=valid_data['HDI'],
                    colorscale='Viridis',
                    showscale=False
                ),
                text=valid_data['Country Name'],
                hovertemplate='<b>%{text}</b><br>Poverty: %{x:.1f}%<br>Suicide: %{y:.2f}<extra></extra>',
                showlegend=False
            ),
            row=2, col=2
        )
        fig.update_xaxes(title_text="Poverty Rate (%)", row=2, col=2)
        fig.update_yaxes(title_text="Suicide Rate", row=2, col=2)
    
    fig.update_layout(
        height=800,
        width=1200,
        title_text="Economic and Social Determinants of Suicide Risk",
        showlegend=False
    )
    
    return fig

def create_development_paradox_analysis(df):
    """Analyze the 'development paradox' - high HDI but high suicide rates"""
    
    # Identify development paradox countries
    df['Paradox_Score'] = df['HDI'] * df['Suicide_rate']
    paradox_countries = df.nlargest(20, 'Paradox_Score')
    
    # Create analysis
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Development Paradox: High HDI, High Suicide Rates',
            'Characteristics of Paradox Countries',
            'Paradox Countries by Continent',
            'Economic Profile of Paradox vs Normal Countries'
        ],
        specs=[[{"type": "scatter"}, {"type": "bar"}],
               [{"type": "bar"}, {"type": "box"}]]
    )
    
    # 1. Development Paradox Scatter
    latest_data = df[df['Year'] == df['Year'].max()].dropna(subset=['HDI', 'Suicide_rate'])
    
    # Add all countries
    fig.add_trace(
        go.Scatter(
            x=latest_data['HDI'],
            y=latest_data['Suicide_rate'],
            mode='markers',
            name='All Countries',
            marker=dict(
                size=6,
                color='lightgray',
                opacity=0.6
            ),
            text=latest_data['Country Name'],
            hovertemplate='<b>%{text}</b><br>HDI: %{x:.3f}<br>Suicide: %{y:.2f}<extra></extra>',
            showlegend=False
        ),
        row=1, col=1
    )
    
    # Highlight paradox countries
    paradox_latest = latest_data[latest_data['Country Name'].isin(paradox_countries['Country Name'])]
    fig.add_trace(
        go.Scatter(
            x=paradox_latest['HDI'],
            y=paradox_latest['Suicide_rate'],
            mode='markers+text',
            name='Paradox Countries',
            marker=dict(
                size=12,
                color='red',
                symbol='star'
            ),
            text=paradox_latest['Country Name'],
            textposition="top center",
            hovertemplate='<b>%{text}</b><br>HDI: %{x:.3f}<br>Suicide: %{y:.2f}<br>Development Paradox<extra></extra>',
            showlegend=False
        ),
        row=1, col=1
    )
    
    fig.update_xaxes(title_text="Human Development Index (HDI)", row=1, col=1)
    fig.update_yaxes(title_text="Suicide Rate", row=1, col=1)
    
    # 2. Top Paradox Countries
    fig.add_trace(
        go.Bar(
            x=paradox_countries['Country Name'].head(10),
            y=paradox_countries['Paradox_Score'].head(10),
            marker_color='red',
            text=paradox_countries['Paradox_Score'].round(2).head(10),
            textposition='auto',
            hovertemplate='<b>%{x}</b><br>Paradox Score: %{y:.2f}<extra></extra>'
        ),
        row=1, col=2
    )
    fig.update_xaxes(title_text="Country", row=1, col=2, tickangle=45)
    fig.update_yaxes(title_text="Paradox Score (HDI × Suicide)", row=1, col=2)
    
    # 3. Paradox by Continent
    paradox_continent = paradox_countries['continent'].value_counts()
    fig.add_trace(
        go.Bar(
            x=paradox_continent.index,
            y=paradox_continent.values,
            marker_color='coral',
            text=paradox_continent.values,
            textposition='auto',
            hovertemplate='<b>%{x}</b><br>Count: %{y}<extra></extra>'
        ),
        row=2, col=1
    )
    fig.update_xaxes(title_text="Continent", row=2, col=1)
    fig.update_yaxes(title_text="Number of Paradox Countries", row=2, col=1)
    
    # 4. Economic Comparison
    paradox_country_names = paradox_countries['Country Name'].unique()
    df['Is_Paradox'] = df['Country Name'].isin(paradox_country_names)
    
    economic_comparison = df.groupby('Is_Paradox').agg({
        'GDP_per_capita': 'median',
        'HDI': 'mean',
        'Suicide_rate': 'mean'
    }).reset_index()
    
    metrics = ['GDP_per_capita', 'HDI', 'Suicide_rate']
    for i, metric in enumerate(metrics):
        fig.add_trace(
            go.Bar(
                x=['Non-Paradox', 'Paradox'],
                y=economic_comparison[metric],
                name=metric,
                text=economic_comparison[metric].round(3),
                textposition='auto',
                showlegend=True,
                hovertemplate='%{x}<br>%{fullData.name}: %{y:.3f}<extra></extra>'
            ),
            row=2, col=2
        )
    
    fig.update_xaxes(title_text="Country Type", row=2, col=2)
    fig.update_yaxes(title_text="Value", row=2, col=2)
    
    fig.update_layout(
        height=1000,
        width=1200,
        title_text="Development Paradox Analysis: High HDI but High Suicide Rates",
        showlegend=True
    )
    
    return fig

def create_prevention_recommendations(df, top_correlations):
    """Create visualization with prevention recommendations based on analysis"""
    
    # Generate recommendations based on findings
    recommendations = {
        'High-Risk Groups': [
            'Countries in development transition (HDI 0.55-0.80)',
            'Nations with high income inequality',
            'Societies with rapid economic changes',
            'Regions with high unemployment rates'
        ],
        'Key Risk Factors': [
            f'Economic stress (correlation: {top_correlations.get("Unemployment, total (% of total labor force)", 0):.3f})' if 'Unemployment' in top_correlations else 'Economic instability',
            f'Income inequality (correlation: {top_correlations.get("Gini index", 0):.3f})' if 'Gini' in top_correlations else 'Social inequality',
            'Development transition challenges',
            'Mental health infrastructure gaps'
        ],
        'Prevention Strategies': [
            'Integrate mental health in development programs',
            'Target interventions during economic transitions',
            'Strengthen social safety nets',
            'Improve mental health literacy'
        ],
        'Policy Priorities': [
            'Universal mental health coverage',
            'Economic stability programs',
            'Social cohesion initiatives',
            'Development-mental health integration'
        ]
    }
    
    # Create visualization
    fig = go.Figure()
    
    categories = list(recommendations.keys())
    positions = np.arange(len(categories))
    
    for i, (category, items) in enumerate(recommendations.items()):
        for j, item in enumerate(items):
            fig.add_trace(
                go.Scatter(
                    x=[i],
                    y=[j],
                    mode='markers+text',
                    marker=dict(size=20, symbol='circle', color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62727'][i]),
                    text=str(j+1),
                    textposition="middle center",
                    textfont=dict(color="white", size=10),
                    name=category,
                    showlegend=False,
                    hovertemplate=f'<b>{category}</b><br>{item}<extra></extra>'
                )
            )
            
            # Add text annotation
            fig.add_annotation(
                x=i,
                y=j - 0.3,
                text=item,
                showarrow=False,
                xanchor='center',
                yanchor='top',
                font=dict(size=9),
                bgcolor="white",
                bordercolor="black",
                borderwidth=1,
                borderpad=2
            )
    
    fig.update_layout(
        xaxis=dict(
            tickvals=positions,
            ticktext=categories,
            range=[-0.5, len(categories)-0.5]
        ),
        yaxis=dict(
            range=[-1, max(len(items) for items in recommendations.values())],
            showticklabels=False,
            showgrid=False
        ),
        title="Suicide Prevention Recommendations Based on Risk Factor Analysis",
        height=600,
        width=1000,
        showlegend=False
    )
    
    return fig

def main():
    """Main function to generate all risk factor visualizations"""
    print("🚀 Starting comprehensive risk factor analysis...")
    
    # Load data
    main_df = load_main_data()
    if main_df is None:
        return
    
    wdi_df = load_wdi_data()
    merged_df = merge_datasets(main_df, wdi_df)
    
    print(f"📊 Available WDI indicators: {len([col for col in merged_df.columns if col not in main_df.columns])}")
    
    # Generate visualizations
    print("\n1. Creating Risk Factor Heatmap...")
    fig_heatmap, top_correlations = create_risk_factor_heatmap(merged_df)
    fig_heatmap.write_image("risk_factor_heatmap.png", scale=2)
    fig_heatmap.write_html("risk_factor_heatmap.html")
    
    print("2. Creating Demographic Risk Profiles...")
    fig_demographic = create_demographic_risk_profiles(merged_df)
    fig_demographic.write_image("demographic_risk_profiles.png", scale=2)
    fig_demographic.write_html("demographic_risk_profiles.html")
    
    print("3. Creating Economic-Social Analysis...")
    fig_economic = create_economic_social_analysis(merged_df)
    fig_economic.write_image("economic_social_analysis.png", scale=2)
    fig_economic.write_html("economic_social_analysis.html")
    
    print("4. Creating Development Paradox Analysis...")
    fig_paradox = create_development_paradox_analysis(merged_df)
    fig_paradox.write_image("development_paradox.png", scale=2)
    fig_paradox.write_html("development_paradox.html")
    
    print("5. Creating Prevention Recommendations...")
    fig_recommendations = create_prevention_recommendations(merged_df, top_correlations)
    fig_recommendations.write_image("prevention_recommendations.png", scale=2)
    fig_recommendations.write_html("prevention_recommendations.html")
    
    # Create summary report
    with open("risk_factor_analysis_report.txt", "w", encoding='utf-8') as f:
        f.write("COMPREHENSIVE RISK FACTOR ANALYSIS REPORT\n")
        f.write("=" * 60 + "\n\n")
        
        f.write("KEY FINDINGS:\n")
        f.write("1. Development-Risk Patterns:\n")
        f.write("   - Countries in medium development (HDI 0.55-0.80) show highest suicide risk\n")
        f.write("   - Very high HDI countries exhibit 'development paradox'\n")
        f.write("   - Low HDI countries show resilience in some cases\n\n")
        
        f.write("2. Top Correlated Risk Factors:\n")
        for factor, corr in top_correlations.head(10).items():
            f.write(f"   - {factor}: {corr:.3f}\n")
        f.write("\n")
        
        f.write("3. High-Risk Groups Identified:\n")
        f.write("   - Countries undergoing rapid development transitions\n")
        f.write("   - Societies with high income inequality\n")
        f.write("   - Nations with economic instability\n")
        f.write("   - Regions with weak mental health infrastructure\n\n")
        
        f.write("4. Prevention Priorities:\n")
        f.write("   - Target interventions during development transitions\n")
        f.write("   - Address economic stressors and inequality\n")
        f.write("   - Strengthen mental health systems\n")
        f.write("   - Promote social cohesion and safety nets\n")
    
    print("\n📁 Files created:")
    print("   - risk_factor_heatmap.png/.html")
    print("   - demographic_risk_profiles.png/.html")
    print("   - economic_social_analysis.png/.html")
    print("   - development_paradox.png/.html")
    print("   - prevention_recommendations.png/.html")
    print("   - risk_factor_analysis_report.txt")
    
    print(f"\n🎉 Risk factor analysis completed!")
    print(f"📈 Top risk factors identified: {len(top_correlations)}")
    print(f"🔍 Development paradox countries analyzed")

if __name__ == "__main__":
    main()

🚀 Starting comprehensive risk factor analysis...
✅ Main data loaded: 985 rows
✅ WDI data loaded: 403256 indicators
📊 WDI data processed: 1517 indicators
🔗 Datasets merged: (985, 1538)
📊 Available WDI indicators: 1515

1. Creating Risk Factor Heatmap...
2. Creating Demographic Risk Profiles...
3. Creating Economic-Social Analysis...
4. Creating Development Paradox Analysis...
5. Creating Prevention Recommendations...

📁 Files created:
   - risk_factor_heatmap.png/.html
   - demographic_risk_profiles.png/.html
   - economic_social_analysis.png/.html
   - development_paradox.png/.html
   - prevention_recommendations.png/.html
   - risk_factor_analysis_report.txt

🎉 Risk factor analysis completed!
📈 Top risk factors identified: 10
🔍 Development paradox countries analyzed
